# Exploring the Data
This notebook to explore and understand the data

In [11]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import numpy as np
from tqdm import tqdm

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=dattlee")
cur = conn.cursor()
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
filepath = "data/song_data"
song_files = get_files(filepath)

In [5]:
df = pd.concat([pd.read_json(song_files[i], lines=True) for i in range(len(song_files))])
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
0,1,AR8ZCNI1187B9A069B,NaN,NaN,,Planet P Project,SOIAZJW12AB01853F1,Pink World,269.81832,1984
0,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007
0,1,AR10USD1187B99F3F1,NaN,NaN,"Burlington, Ontario, Canada",Tweeterfriendly Music,SOHKNRJ12A6701D1F8,Drop of Rain,189.57016,0
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


In [6]:
df.isnull().sum(axis = 0)

num_songs            0
artist_id            0
artist_latitude     41
artist_longitude    41
artist_location      0
artist_name          0
song_id              0
title                0
duration             0
year                 0
dtype: int64

Here we see only lat and long have null values

In [7]:
def find_min_max_len(column):
    data_type = type(column[0])
    
    minimum = float('inf')
    maximum = float('-inf')
    
    if data_type == str:
        for i in column:
            if i is None:
                continue
            if len(i) < minimum:
                minimum = len(i)
            elif len(i) > maximum:
                maximum = len(i)
    elif data_type == int or data_type == np.int32:
        for i in column:
            if i < minimum:
                minimum = i
            elif i > maximum:
                maximum = i
    elif data_type == float:
        precision = 0
        scale = 0
        for i in column:
            if pd.isna(i):
                continue
            p, s = str(i).split('.')
            if len(p) > precision:
                precision = len(p)
            if len(s) > scale:
                scale = len(s)
                
            if i < minimum:
                minimum = i
            elif i > maximum:
                maximum = i
        print('max scale {}\n max precision {}'.format(scale, scale+precision, ))
        
    
    print('minimum', minimum, '\nmaximum', maximum)

In [9]:
song_id = 6
title = 7
artist_id = 1
year = 9
duration = 8
song_data = df.to_numpy()[:,[song_id, title, artist_id, year, duration]]

In [12]:
print('song id')
find_min_max_len(song_data[:,[0]].flatten())

print('\ntitle')
find_min_max_len(song_data[:,[1]].flatten())

print('\nartist id')
find_min_max_len(song_data[:,[2]].flatten())

print('\nyear')
find_min_max_len(song_data[:,[3]].flatten())

print('\nduration')
find_min_max_len(song_data[:,[4]].flatten())

song id
minimum 18 
maximum 18

title
minimum 5 
maximum 52

artist id
minimum 18 
maximum 18

year
minimum 0 
maximum 2008

duration
max scale 5
 max precision 8
minimum 29.54404 
maximum 599.24853


In [ ]:
users_df['userId'] = users_df['userId'].astype('int')